In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

#DATOS_DIR = '~/buckets/b1/datasets/'
DATOS_DIR = '../data/'  
df_final = pd.read_parquet(DATOS_DIR+'FE_06_dataset.parquet') 
df_final.columns = df_final.columns.str.replace(' ', '_').str.replace(r'[^A-Za-z0-9_]', '', regex=True)
df_true = pd.read_csv(DATOS_DIR+'pred/TRUE-DECEMBER.csv') 
pred = pd.read_csv(DATOS_DIR+'pred/PRED-DECEMBER.csv') 


# Análisis de predicciones de octubre para diciembre

In [2]:
# Merge the dataframes on 'product_id'
merged_df = df_true.merge(pred, on='product_id', suffixes=('_true', '_pred'))


# Create a line plot with Plotly
fig = go.Figure()

# Add true TN line
fig.add_trace(go.Scatter(x=merged_df['product_id'], y=merged_df['tn_true'], 
                         mode='lines+markers', name='True TN'))

# Add predicted TN line
fig.add_trace(go.Scatter(x=merged_df['product_id'], y=merged_df['tn_pred'], 
                         mode='lines+markers', name='Predicted TN'))

# Update layout
fig.update_layout(
    title='True vs Predicted TN',
    xaxis_title='Product ID',
    yaxis_title='TN',
    legend_title='Series',
    hovermode='x unified'
)

# Show the plot
fig.show()

In [3]:

# Calculate the difference
merged_df['difference'] = abs(merged_df['tn_true'] - merged_df['tn_pred'])

# Create a line plot with Plotly
fig = go.Figure()

# Add the difference line
fig.add_trace(go.Scatter(x=merged_df['product_id'], y=merged_df['difference'], 
                         mode='lines+markers', name='Difference (True - Pred)'))

# Update layout
fig.update_layout(
    title='Difference between True and Predicted TN',
    xaxis_title='Product ID',
    yaxis_title='Difference (True - Pred)',
    legend_title='Series',
    hovermode='x unified'
)

# Show the plot
fig.show()

# Análisis de productos/clientes

In [4]:
prod_id = 20072
col = 'weight'

# Pivot the dataframe to have customer_id as columns and periodo as the index
df_filtered_product = df_final.copy()
df_filtered_product =  df_filtered_product[df_filtered_product['product_id'] == prod_id]


df_filtered_product.reset_index(inplace=True)
df_pivot = df_filtered_product.pivot(index='periodo', columns='customer_id', values= col)

# Convert 'periodo' from PeriodIndex to Timestamp
df_pivot.index = df_pivot.index.to_timestamp()

# Create the plot using Plotly
fig = go.Figure()

for column in df_pivot.columns:
    fig.add_trace(go.Scatter(x=df_pivot.index, y=df_pivot[column], mode='lines', name=f'Customer {column}'))

# Customize the layout
fig.update_layout(
    title=f'TN Series for (Product ID: {prod_id})',
    xaxis_title='Period',
    yaxis_title=col,
    hovermode='x unified'
)

# Show the plot
fig.show()

In [13]:
prod_id = 20072
col = 'weight'

# Pivot the dataframe to have customer_id as columns and periodo as the index
df_filtered_product = df_final.copy()
df_filtered_product =  df_filtered_product[(df_filtered_product['product_id'] == prod_id)]
df_filtered_product.reset_index(inplace=True)
# Find the customer_id with the maximum value of the column col
max_customer_id = df_filtered_product.loc[df_filtered_product[col].idxmax(), 'customer_id']

print(f'The customer_id with the maximum value in the column {col} is {max_customer_id}')



df_filtered_product =  df_filtered_product[(df_filtered_product['customer_id'] == max_customer_id)]



df_filtered_product.reset_index(inplace=True)
df_pivot = df_filtered_product.pivot(index='periodo', columns='customer_id', values= col)

# Convert 'periodo' from PeriodIndex to Timestamp
df_pivot.index = df_pivot.index.to_timestamp()

#Create the plot using Plotly
fig = go.Figure()

for column in df_pivot.columns:
    fig.add_trace(go.Scatter(x=df_pivot.index, y=df_pivot[column], mode='lines', name=f'Customer {column}'))

# Customize the layout
fig.update_layout(
    title=f'TN Series for Customer {max_customer_id} (Product ID: {prod_id})',
    xaxis_title='Period',
    yaxis_title=col,
    hovermode='x unified'
)

# Show the plot
fig.show()

The customer_id with the maximum value in the column weight is 10004
